In [6]:
import google.generativeai as genai
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()
genai.configure(api_key=os.environ['GEMINI_KEY'])

True

In [ ]:
model = genai.GenerativeModel(model_name="gemini-1.5-flash")
model.generate_content()